# Lab 07

## Zadanie 1

Przygotuj demo programu znajdującego miejsca zerowe metodą Newtona. Wykorzystując `argparse` ([link](https://docs.python.org/3.6/library/argparse.html#module-argparse)) lub `optparse` ([link](https://docs.python.org/3.6/library/optparse.html)) obsłuż:
- ustalane punktu startowego,
- wielkość kroku w pochodnej,
- ilość kroków metody,
- dokładność
- pomoc

Program uruchamiamy podając, np.:

`./newton.py x**2+x+1 -h 0.00001`

In [ ]:
import argparse
from scipy.misc import derivative


def newton_method(function, initial_guess, derivative_step, max_steps, precision):
    x = initial_guess
    steps = 0
    while steps < max_steps:
        f = function(x)
        f_prime = derivative(function, x, dx=derivative_step)
        if abs(f_prime) < precision:
            break
        x = x - f / f_prime
        steps += 1
    return x


def main():
    parser = argparse.ArgumentParser(description='Demo program finding zero points using Newton\'s method')
    parser.add_argument('function', type=str, help='Function for finding zero points')
    parser.add_argument('-i', '--initial_guess', type=float, default=0, help='Initial guess (default: 0)')
    parser.add_argument('-s', '--derivative_step', type=float, default=0.001,
                        help='Step size in derivative calculation (default: 0.001)')
    parser.add_argument('-m', '--max_steps', type=int, default=100,
                        help='Maximum number of method steps (default: 100)')
    parser.add_argument('-p', '--precision', type=float, default=0.0001, help='Precision (default: 0.0001)')

    args = parser.parse_args()
    function = lambda x: eval(args.function)

    result = newton_method(function, args.initial_guess, args.derivative_step, args.max_steps, args.precision)

    print(f"Zero point of function {args.function} found using Newton's method: {result}")


if __name__ == '__main__':
    main()

## Zadanie 2

Zadanie należy wykonać wykorzystując program `BagOfWords` z poprzednich zajęć.


Przerób ją tak, żeby interpunkcja, cyfry i wszelkie inne znaki nie przeszkadzały w parsowaniu tekstu. Uruchom ja na [tekście hamleta](http://www.gutenberg.org/cache/epub/1787/pg1787.txt). Ile razy występuje słowo hamlet? Jak brzmi 10 najczęściej występujących słów?

In [31]:
import re
from collections import defaultdict
from typing import IO, Union, Optional
import io
import functools

class BagOfWords:
    def __init__(self, content: Optional[Union[str, IO]] = None):
        self.d = defaultdict(int)
        pattern = re.compile('[,\.\d;\[\]?!]')
        if isinstance(content, io.TextIOWrapper):
            for line in content:
                line = pattern.sub("", line)
                for i in line.split():
                    i = re.sub(r"\W?(\w+(\W?\w+)*)\W?", lambda x: x[1], i)
                    i = i.lower()
                    self.d[i] += 1
            content.close()
        else:
            content = pattern.sub("", content)
            for i in content.split():
                # catches asdf- etc. and swaps for asdf, leaves asdf-zxcv
                print(i)
                i = re.sub(r"\W?(\w+(\W?\w+)*)\W?", lambda x: x[1], i)
                print(i)
                i = i.lower()
                self.d[i] += 1

    def __iter__(self):
        sorted_keys = sorted(self.d.items(), key=lambda item: item[1], reverse=True)
        return iter(sorted_keys)

    def __str__(self):
        return functools.reduce(lambda init, x: str(init) + " " + str(x), self.__iter__())

    def __repr__(self):
        return self.__str__()

    def __contains__(self, item):
        try:
            _ = self.d[item]
            return True
        except KeyError:
            return False

    def __getitem__(self, item):
        return self.d[item]

    def __setitem__(self, key, value):
        self.d[key] = value

    def __add__(self, other):
        new_bag = BagOfWords('')
        for word in self.d:
            new_bag.d[word] += self.d[word]
        for word in other.d:
            new_bag.d[word] += other.d[word]
        return new_bag

s = """ So to seduce!- won to his shameful lust
    The will of my most seeming-virtuous queen.
    O Hamlet, what a falling-off was there,"""

b = BagOfWords(open("lab07_files/hamlet.txt"))
# b = BagOfWords("-")
print(b)

('the', 1099) ('and', 991) ('to', 742) ('of', 702) ('i', 575) ('a', 558) ('you', 554) ('my', 520) ('in', 434) ('it', 419) ('that', 398) ('is', 363) ('ham', 358) ('not', 324) ('his', 304) ('this', 301) ('with', 286) ('for', 278) ('but', 274) ('your', 251) ('as', 237) ('me', 234) ('be', 232) ('lord', 225) ('he', 213) ('what', 208) ('so', 205) ('him', 197) ('king', 196) ('have', 183) ('will', 172) ('do', 151) ('are', 150) ('or', 142) ('no', 142) ('we', 140) ('by', 138) ('on', 129) ('o', 124) ('all', 123) ('our', 120) ('queen', 119) ('shall', 115) ('if', 114) ('good', 109) ('hor', 109) ('hamlet', 108) ('come', 107) ('thou', 105) ('they', 101) ('now', 98) ('let', 96) ('more', 96) ('from', 95) ('her', 91) ('how', 90) ('at', 87) ('thy', 86) ('pol', 86) ('was', 84) ('most', 82) ('like', 81) ('would', 81) ('may', 80) ('there', 77) ('well', 77) ('them', 75) ('sir', 75) ('know', 74) ('enter', 73) ('tis', 72) ('th', 71) ('go', 71) ('us', 70) ('did', 66) ('very', 66) ('love', 66) ('then', 65) ('hat

e

## Zadanie 3

Wykorzystując `pickle` zapisz i odczytaj klasy z poprzedniego zadania nakarmonej Hamletem. Porównaj metody i rozmiar.

In [37]:
import pickle

b = BagOfWords(open("lab07_files/hamlet.txt"))
print(len(b.__str__()))
pickled = pickle.dumps(b)
print(len(pickled))
unpickled = pickle.loads(pickled)


70747
55850


## Zadanie 4

Wykorzystując [https://gist.github.com/pamelafox/986163](https://gist.github.com/pamelafox/986163) podaj aktualną godzinę we:
- wszystkich krajach, wyświetlając je zgrupowane względem kontynentów,
- przeprowadź symulacyjne wyświetlanie kolejnych krajów w miarę jak w danym kraju wybija północ, opóżnienie wyświetlania ustaw proporcjonalne do realnego czasu

In [79]:
import ast
import requests
import itertools

x = requests.request("GET", "https://gist.githubusercontent.com/pamelafox/986163/raw/eeeabff229831334f8a84bdddd5768cc84c50bd4/countryinfo.py").text
# s = json.loads()
# print(countries)

x = x.replace('countries = ', '')
countries_dict = ast.literal_eval(x)
countries_dict = countries_dict[0:10]
i = 0
for k in countries_dict:
    print(f"{k}")
    i += 1
    if i == 10: break
# print(type(countries_dict[0]))
# print(countries_dict[0]["continent"])
grouped = itertools.groupby(countries_dict, lambda country: country["continent"])
# print(list(grouped))
# print(grouped[0][1])
# print(type(grouped))
# print(type(list(grouped)))
# print(len(list(grouped)))
# countries_dict
i = 0
for k, v in grouped:
    print(f"{k}: {list(v)}")
    i += 1
    if i == 10: break

{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'}
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'}
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"}
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'}
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'}
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'}
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Us

## Zadanie 5

Dla klasy `BagOfWords` napisz metody `save` oraz `load` wykorzystujące `json`'a do zapisu i odczytu danych.